# **STEMfit example**
## *Gaussian fitting mode*

We start by importing `STEMfit` and loading the image, the filtered image and initial atomic positions we got from the center of mass mode:

In [ ]:
import STEMfit

In [ ]:
image = STEMfit.load_image("image.tif", convert=true);
filtered_image = STEMfit.load_image("Center of Mass results\\filtered image.png", convert=true);
atom_parameters = STEMfit.load_atomic_parameters("Center of Mass results\\atomic parameters.csv");

`STEMfit` will now try to find potential unit cells in the atomic positions. To do this, it finds clusters in the vectors between each pair of atoms using the [DBSCAN](https://www.wikiwand.com/en/DBSCAN) algorithm. Any two of such vectors may form the side of a possible unit cell. We restrict the angle of the unit cell to 85-95 degrees (i.e. a rectangular unit cell) using the `uc_allowed_angles` parameter and use the default settings for all other parameters. The possible choices for unit cells are automatically plotted together with the average *local* atomic environment.

In [ ]:
(unit_cells, neighbors, atom_tree) = STEMfit.find_unit_cells(atom_parameters, uc_allowed_angles=85:95);

It seems like `STEMfit` found a correct unit cell (Unit cell 3). Bear in mind that this unit cell represents the *average* translational symmetry in the whole image. We use this unit cell to set up a model of the image which we will try to fit. First, we determine the background contribution:

In [ ]:
background_image = STEMfit.construct_background(filtered_image, unit_cells[3]);

We then obtain initial parameters for the gaussians using the atomic parameters we calculated using the center of mass mode:

In [ ]:
fitting_parameters = STEMfit.fitting_parameters(atom_parameters_fitted, background_image);

Finally, we set up the model and fit it to the image. This can take a few minutes. `STEMfit` first performs a global optimization of the atom widths, since these depend on the thresholding parameters chosen in the center of mass mode. Then, it optimized the parameters of each of the atoms individually. 

In [ ]:
image_model = STEMfit.ImageModel(fitting_parameters, unit_cells[3], background_image);

In [ ]:
STEMfit.fit!(image_model, image)

Let's look at the results. On the left we display the experimental data, in the middle is the fitted model and on the right the residual image of RMS differences between the image and the fitted model. 

In [ ]:
simulated_image = STEMfit.produce_image(image_model)
residual_image = STEMfit.residual_image(image, simulated_image)
STEMfit.show_images(image, simulated_image, residual_image)

An excellent fit has been achieved, so we save the results:

In [ ]:
STEMfit.save_model(image_model, "Gaussian fitting results\\atomic parameters.csv")
STEMfit.save("Gaussian fitting results\\simulated image.png", simulated_image)
STEMfit.save("Gaussian fitting results\\residual image.png", residual_image)